In [ ]:
#Importing Libraries
import warnings
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.core.display import HTML

warnings.filterwarnings('ignore') # if there are any warning due to version mismatch, it will be ignored
#Reading Csv File
loan= pd.read_csv('loan.csv')
loan.head(5)


In [ ]:
loan.info()

In [ ]:
loan.shape


In [ ]:
#Data Cleaning
nullcolumns = loan.isnull().sum()
#print(nullcolumns)
filterhighnullvalues=nullcolumns.loc[(nullcolumns.values>30000)]
filterhighnullvalues.head
#Observations : Below columns are having null values 

In [ ]:
#Columns having more null values are not required for analysis. Droping those column
loan.drop(columns=list(filterhighnullvalues.index), inplace=True)
loan.shape

In [ ]:
#Finding the unique values
Uniquecol=loan.nunique()
singlevaluecolumn=Uniquecol.loc[Uniquecol.values==1]
#No unique values means we can drop them as they are not useful for analysis
loan.drop(columns=list(singlevaluecolumn.index),inplace=True)
loan.shape

In [ ]:
loan.info()

In [ ]:
#Removing unwanted columns for analysis
del_cols = ["id","member_id","url","zip_code","out_prncp","out_prncp_inv","total_pymnt","total_pymnt_inv",
            "total_rec_prncp","total_rec_int","total_rec_late_fee","recoveries","collection_recovery_fee",
            "last_pymnt_d","last_pymnt_amnt","last_credit_pull_d"]
loan.drop(columns=del_cols, inplace=True)
loan.shape

In [ ]:
loan.info()

In [ ]:
loan['loan_status'].value_counts()


In [ ]:
#current loan status is not taken into consideration for the analysis
loan=loan[~(loan['loan_status'] == 'Current')]
loan.shape

In [ ]:
loan['loan_status'].value_counts()


In [ ]:
#Looking into term and interest rate for analysis
loan['term'].value_counts()


In [ ]:
loan['int_rate'].head()


In [ ]:
#Removing percent symbol from the interest for calculation
loan['int_rate'] = loan['int_rate'].apply(lambda x: pd.to_numeric(x.split("%")[0]))
loan['int_rate'].describe()

In [ ]:
loan.info()

In [ ]:
loan.annual_inc.value_counts().sort_index(ascending=False)

In [ ]:
loan['sub_grade'].value_counts()

In [ ]:
print(loan['emp_title'].value_counts().head())
loan['emp_length'].value_counts().head()
loan['home_ownership'].value_counts()
loan['verification_status'].value_counts()
loan['purpose'].value_counts()


In [ ]:
loan['title'].nunique()
loan['title'].sample(10)

In [ ]:
#Title has too many unique  values which is not required for analysis
#loan.drop(columns=['title'], inplace=True)
loan.shape

In [ ]:
loan['earliest_cr_line'].nunique()

In [ ]:
from datetime import datetime
today = datetime.today()
today.year
def days_from_earliest_cr_line (d):
    m, y = d.split('-')
    if y in ['Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sep','Oct','Nov','Dec']:
        y, m = d.split('-')
    y = 2000 + int(y)
    if y > today.year:
        y -= 100
    earliest_cr_line = datetime.strptime(str(y)+'-'+m+'-1', '%Y-%b-%d')
    return (today - earliest_cr_line).days
days_from_earliest_cr_line("Dec-20")

In [ ]:
loan['days_from_earliest_cr_line'] = loan['earliest_cr_line'].apply(days_from_earliest_cr_line)
loan['days_from_earliest_cr_line'].describe()


In [1515]:
#earliest_cr_line -The month the borrower's earliest reported credit line was opened which is not required for analysis
#so drop this column

In [1516]:
loan.drop(columns=['earliest_cr_line'],inplace=True)
loan.drop(columns=['title'], inplace=True)


In [ ]:
loan.columns

In [ ]:
loan.drop(columns=['days_from_earliest_cr_line'],inplace=True)
loan.columns

In [1519]:
#Data Conversion

In [ ]:
loan.info()

In [ ]:
#converting analysis int values to float
loan = loan.astype({
    'dti': 'float',
    'funded_amnt' : 'float',
    'funded_amnt_inv' : 'float',
    'loan_amnt' : 'float'
})
loan.info()

In [ ]:
#Checking the data in the term column 
loan['term'] 

#converting term into int
loan['term']=loan['term'].apply(lambda x:int(x.replace('months','').strip()))
loan['term']

In [ ]:
# Rounding of the values of below columns to two decimal places
for col in ['loan_amnt', 'funded_amnt', 'funded_amnt_inv', 'int_rate', 'dti']:
    loan[col] = loan[col].apply(lambda x: round(x, 2))
loan

In [1524]:
#Common Graph functions
#--Function Desc--
#Generate Box plot based on the column names in the dataframe.
'''  Parameters:
        df (pd.DataFrame): The DataFrame containing the data.
        column_name (str): The name of the column to be plotted.
        title (str, optional): The title of the plot. Default is None.
        orient (str): The orientation of the plot. Default is horizontal. Possible values - h, v.'''
def Generate_boxplot(df: pd.DataFrame, column_name: str, title: str = None, horizontal: bool = True) -> None:
        if column_name not in df.columns:
            print(f"Error: Column '{column_name}' not found in the DataFrame.")
            return
        plt.figure(figsize=(8, 6))
        if horizontal:
            sns.boxplot(y=df[column_name], orient='h', palette='Set2')
        else:
            sns.boxplot(x=df[column_name], orient='v', palette='Set2')
        if title:
            plt.title(title)
        plt.xlabel(column_name, fontsize=12)
        plt.ylabel('Values', fontsize=12)
        plt.show()
   


In [1525]:
def calculate_boxplot_stats(df: pd.DataFrame, column: str, iqr_multiplier: float = 1.5) -> dict:
    """
    Calculate the Interquartile Range (IQR) and the lower and upper bounds for outlier detection.

    Parameters:
        df (pd.DataFrame): The DataFrame containing the data.
        column (str): The name of the column for which to calculate the IQR and bounds.
        iqr_multiplier (float, optional): Multiplier for the IQR to determine bounds. Default is 1.5.

    Returns:
        dict: A dictionary containing the following values:
            - 'quartile1': The first quartile (25th percentile).
            - 'quartile3': The third quartile (75th percentile).
            - 'iqr': The Interquartile Range (IQR).
            - 'lower_bound': The lower bound for outlier detection.
            - 'upper_bound': The upper bound for outlier detection.
    """
    quartile1 = df[column].quantile(0.25)
    quartile3 = df[column].quantile(0.75)
    iqr = quartile3 - quartile1
    lower_bound = quartile1 - iqr_multiplier * iqr
    upper_bound = quartile3 + iqr_multiplier * iqr
    return {
        'iqr': iqr,
        'quartile1': quartile1,
        'quartile3': quartile3,
        'lower_bound': lower_bound,
        'upper_bound': upper_bound
    }


In [ ]:
# Calculate the lower bound and upper bound
is_annual_inc_lb = calculate_boxplot_stats(loan, 'annual_inc')['lower_bound']
is_annual_inc_ub = calculate_boxplot_stats(loan, 'annual_inc')['upper_bound']
is_annual_inc_lt = loan.annual_inc > is_annual_inc_lb
is_annual_inc_gt = loan.annual_inc < is_annual_inc_ub

req_income = is_annual_inc_lt & is_annual_inc_gt

# Filter the rows with required annual income range between lower and upper bound
loan_reqincome = loan[req_income]

# Show boxplot
Generate_boxplot(loan_reqincome, 'annual_inc', title='Annual Income')

In [ ]:
for col in ['loan_amnt', 'funded_amnt', 'funded_amnt_inv', 'int_rate', 'installment', 'dti']:
    Generate_boxplot(loan, col, horizontal=False)


In [ ]:
loan['emp_title'].nunique
loan.drop(columns=['emp_title'],inplace=True)
loan.info()

In [ ]:
#Analysing the grade and subgrade columns 
loan['grade'].value_counts()
loan['sub_grade'].value_counts()

In [ ]:
#Checking revol_util column 
loan['revol_util'].head()


In [1531]:
#converting revol_util into  numerical values
loan['revol_util'] = pd.Series(loan['revol_util']).str.replace('%', '').astype(float)



In [ ]:
loan['revol_util']

In [1533]:
loan['issue_d'] = pd.to_datetime(loan['issue_d'], format = '%b-%y')

In [ ]:
loan.info()

In [ ]:
loan['loan_status'].value_counts()

In [ ]:
loan_status_percent = loan['loan_status'].value_counts(normalize=True)
loan_status_percent

In [ ]:
loan.head()

In [ ]:
loan.isna().sum()

In [ ]:
#Univariate Analysis

# changing the loan_status to a numeric variable, assign 1 for defaulted loans and 0 for paid off ones
loan['loan_status_num'] = loan['loan_status'].apply(lambda x: 1 if x=='Charged Off' else 0)

loan.dropna(axis = 1, how = 'all', inplace = True)
loan.head()



In [ ]:
sns.set_style("whitegrid")
sns.clustermap(data=loan.corr(numeric_only=True),
    figsize=(20, 10))
plt.show()

In [1541]:
#using the above diagram we can able to find the high correlation between Installment,funded_amnt,loan_amnt
# and funded_amnt_inv. Next Coorelation between pub_rec & pub_rec_bankrupcies and number of accounts

In [ ]:
# here are some of the global settings/constants that will be referred to within the functions defined below
#colors = ["#fca9a9","#a9fea9"]
colors = ["#a9fea9","#fca9a9"]
sns.set(style='whitegrid', palette=sns.color_palette(colors))
by = 'loan_status'
order = ['Fully Paid','Charged Off']
# loan_status_pct from above will be used in the below functions as well
plt.figure(figsize=(20, 12))
plt.subplot(2,2,1)
sns.boxplot(x = 'loan_status', y = 'loan_amnt', data = loan)
plt.show()

In [ ]:
loan['loan_amnt'].describe()


In [1673]:
#Generic Method for univariant analysis 

def CalculatePercentile (*, inpdata=loan, column,intrelated=''):
    """
   This Method will returns the column name with 
           Method Params:
            data:Dataframe
            column:this is the continuous numeric field which has to be binned to its percentiles
  """
    
    new_column = column + '_perc'
    Perc_labels = ['Low ','Lower ','Medium ','High ','Highest ']
    Perc_labels = [Perc_labels[p] + '(' +
              str(round(np.nanpercentile(inpdata[column], p * 20),2)) +
              intrelated + ' - ' +
              str(round(np.nanpercentile(inpdata[column], (p+1) * 20),2)) +
              intrelated + ')'
              for p in range(5)]
    inpdata[new_column] = pd.qcut(inpdata[column], q=5, labels=Perc_labels)


#Function for Univariate Categorical
def GenericMethod_Univariant (*, data=loan, column, title=None, figsize=(10,7),  box_plot=False, datafilter=None):

    plotdata = data if datafilter is None else data.loc[datafilter]

    fig, ax = plt.subplots(figsize=figsize)
    sns.countplot(data=plotdata, y=column, hue=by, ax=ax,palette=['#432371',"#7fcdbb"] ,
                      order=plotdata[column].value_counts().index, hue_order=order)
    if title is not None:
        plt.title(title)
        plt.show()
        dataframe = loan.groupby([column])[by].value_counts(normalize=True).rename('Charged off %').reset_index().sort_values(column)
        dataframe = dataframe.loc[dataframe[by]=='Charged Off'].sort_values(by='Charged off %', ascending=False).drop(columns=[by])
        df2 = loan.groupby([column])[by].count()
        dataframe = dataframe.merge(right=df2,on=column).rename(columns={by:'Record count'})
        display(HTML(dataframe.to_html(index=False)))
    if box_plot:
        df2.plot(kind='box',logy=True,x="No. of records of defaulted loans per category")
        plt.show()    


In [ ]:
# Based on relationals we can now analyse highly coorelated items 
CalculatePercentile(column='loan_amnt')
GenericMethod_Univariant(column='loan_amnt_perc',title="Analysis by Loan Amount",box_plot=True)


In [ ]:

CalculatePercentile(column='int_rate')
GenericMethod_Univariant(column='int_rate_perc',title="Analysis by Interest")

In [ ]:
CalculatePercentile(column='annual_inc')
GenericMethod_Univariant(column='annual_inc_perc',title="Analysis by Annual Income")

In [ ]:
GenericMethod_Univariant(column='term')

In [ ]:
Analysis Based on Loan Amount,Interest ,Annual Income,Term:
* Higher the Loan Amount and interest rate higher the charged Off.
* Lower Tenure  results in higher charged Off.
* Lower Annual Income results in higher charged Off.


In [ ]:
GenericMethod_Univariant(column='purpose')
GenericMethod_Univariant(column='verification_status')

In [ ]:
Major Impact Cases Based on Univariant Analysis
1. Debt Consolidation 
2. Verification Status

In [ ]:
Univariate_Categorical(column='emp_length')


In [ ]:
Univariate_Categorical(column='home_ownership')

In [ ]:
Univariate_Categorical(column='pub_rec_bankruptcies', box_plot=True)

In [ ]:
PercentileCut(column='dti')
Univariate_Categorical(column='dti_p')

In [ ]:
#Bivariate Analysis

def Bivariate_Analysis (*, data=loan, x, y, title=None, figsize=(8,6)):
    """
        Parameters:
            data : defaulted at the 'loan' variable, but can be changed externally if needed
            x, y : columns of data in the corresponding axis, one numerical and one categorical
            figsize : a default have is given, but can be overridden
        Returns:
            None, does a violin plot
    """
    fig, ax = plt.subplots(figsize=figsize)
    sns.violinplot(data=data, x=x, y=y, hue=by, split=True, figsize=figsize,
                   ax=ax, hue_order=order)
    if title is not None:
        plt.title(title)
    plt.show()

def Bivariate_Categorical (*, data=loan, x, y, title=None, figsize=(8,6)):
    """
        Parameters:
            data : defaulted at the 'loan' variable, but can be changed externally if needed
            x, y : columns of data in the corresponding axis, both categorical
            figsize : a default have is given, but can be overridden
        Returns:
            None, does a heatmap plot
    """
    fig, ax = plt.subplots(figsize=figsize)
    pt = pd.pivot_table(data=data, values='loan_status_num', index=y, columns=x)
    sns.heatmap(pt, ax=ax, cmap='YlGnBu')
    if title is not None:
        plt.title(title)
    plt.show()




In [ ]:
Bivariate_Categorical(x='purpose', y='annual_inc_p', figsize=(10,4))

In [ ]:

Bivariate_Categorical(x='dti_p', y='annual_inc_p', figsize=(10,4))